# Try reading ScienceBase DEM as COG on S3
This is an experiment to demonstrate the value of putting ScienceBase item on S3 in a cloud-optimized format. 

I took the 230GB `.tar.bz2` file on S3 from https://www.sciencebase.gov/catalog/item/5af335b2e4b0da30c1b26dab, converted it to 240GB Cloud Optimized GeoTIFF (COG), and put it back on S3. Here's a simple example of reading/visualizing one of the COG overlays using xarray, rasterio and holoviz tools.

In [17]:
import rasterio
import xarray as xr
import hvplot.xarray
import geoviews as gv

import rioxarray as rx
import os

DEM data is stored as COG on S3

In [18]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

In [19]:
cog_url = 's3://pangeo-data-uswest2/esip/sciencebase/Southern_California_Topobathy_DEM_1m_cog.tif'

In [20]:
da = xr.open_rasterio(cog_url);

In [21]:
da.nbytes/1e6

284782.39216

Check out the overviews:

In [22]:
src = rasterio.open(cog_url, 'r')
[src.overviews(i) for i in src.indexes]

[[4, 8, 16, 32, 64, 128, 256, 512, 1023]]

Pick an overview and read directly from that:

In [32]:
da = rx.open_rasterio(cog_url, masked=True, overview_level=7)

Use rioxarray to get CRS

In [33]:
da.nbytes/1e6

2.17536

In [25]:
crs = da.rio.crs.to_epsg()
crs

26911

Plot up the bathy with a tile service basemap

In [26]:
gv.tile_sources.OSM  * da.hvplot.quadmesh(rasterize=True, crs=crs, colormap='viridis').opts(alpha=0.7)

/srv/conda/envs/notebook/lib/python3.7/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not
/srv/conda/envs/notebook/lib/python3.7/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not


/srv/conda/envs/notebook/lib/python3.7/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not


:DynamicMap   [band]
   :Overlay
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .Image.I :Image   [x,y]   (value)

Ocean bathy are values less below sea level from 1st band:

In [27]:
ocean = da.where(da<0)[0,:,:]

Try hill shaded relief using datashader's [geoprocessing routines](http://datashader.org/user_guide/8_Geography.html) 

In [10]:
import datashader.geo as dsgeo

/srv/conda/envs/notebook/lib/python3.7/site-packages/datashader/geo.py:28: VisibleDeprecationWarning: The datashader.geo module is deprecated as of version 0.11.0. Its contents have migrated to the xarray_spatial library, github.com/makepath/xarray-spatial.
  VisibleDeprecationWarning


In [11]:
ocean_hs = dsgeo.hillshade(ocean)

In [12]:
gv.tile_sources.OSM  * ocean.copy(data=ocean_hs.data).hvplot.image(rasterize=True, crs=crs, colormap='gray')

/srv/conda/envs/notebook/lib/python3.7/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not


:DynamicMap   []
   :Overlay
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .Image.I :Image   [x,y]   (value)